In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")



In [3]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [4]:
df_2 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum")).reset_index()

In [19]:
df_2.reset_index(drop=True)

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,Anzahl_registered,Anteil_registered,weekend,date_delta
0,2020-01-04,985,16.489909,79.678173,16.226271,0.0,0.0,654,0.663959,True,0
1,2020-01-05,801,14.415006,67.141074,15.875136,0.0,0.0,670,0.836454,True,1
2,2020-01-06,1349,8.440104,40.287620,14.724723,1.0,0.0,1229,0.911045,False,2
3,2020-01-07,1562,9.051498,56.411012,12.351703,1.0,0.0,1454,0.930858,False,3
4,2020-01-08,1600,9.649350,38.230000,14.113182,1.0,0.0,1518,0.94875,False,4
...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,4231,10.475495,59.472229,22.242182,1.0,0.0,4024,0.951075,False,726
727,2021-12-31,8144,10.419155,58.653487,10.341263,1.0,0.0,7629,0.936763,False,727
728,2022-01-01,5932,10.551763,76.322151,8.237662,0.0,0.0,5293,0.892279,True,728
729,2022-01-02,5182,10.778680,46.243342,23.974396,0.0,0.0,4392,0.847549,True,729


In [20]:
df_2.dtypes

datetime             datetime64[ns]
Anzahl_pro_Tag                int64
temp                        float64
humidity                    float64
windspeed                   float64
workingday                  float64
holiday                     float64
Anzahl_registered             Int64
Anteil_registered           Float64
weekend                        bool
date_delta                    int64
dtype: object

In [7]:
df_2["workingday"] = df_2["workingday"].round()

In [8]:

df_2["holiday"] = df_2["holiday"].round() 

In [9]:
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]

In [10]:
df_2["weekend"] = df_2["datetime"].dt.weekday > 4

In [11]:
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days

In [12]:
df_2.drop("Anzahl_registered",axis=1)

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,Anteil_registered,weekend,date_delta
0,2020-01-04,985,16.489909,79.678173,16.226271,0.0,0.0,0.663959,True,0
1,2020-01-05,801,14.415006,67.141074,15.875136,0.0,0.0,0.836454,True,1
2,2020-01-06,1349,8.440104,40.287620,14.724723,1.0,0.0,0.911045,False,2
3,2020-01-07,1562,9.051498,56.411012,12.351703,1.0,0.0,0.930858,False,3
4,2020-01-08,1600,9.649350,38.230000,14.113182,1.0,0.0,0.94875,False,4
...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,4231,10.475495,59.472229,22.242182,1.0,0.0,0.951075,False,726
727,2021-12-31,8144,10.419155,58.653487,10.341263,1.0,0.0,0.936763,False,727
728,2022-01-01,5932,10.551763,76.322151,8.237662,0.0,0.0,0.892279,True,728
729,2022-01-02,5182,10.778680,46.243342,23.974396,0.0,0.0,0.847549,True,729


In [22]:
#trainings-Test-Split
features = ["date_delta", 'temp', 'humidity','windspeed',"workingday","holiday","Anteil_registered","weekend"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

In [21]:
X

,date_delta,temp,humidity,windspeed,workingday,holiday,Anteil_registered,weekend
0,0,16.489909,79.678173,16.226271,0.0,0.0,0.663959,True
1,1,14.415006,67.141074,15.875136,0.0,0.0,0.836454,True
2,2,8.440104,40.287620,14.724723,1.0,0.0,0.911045,False
3,3,9.051498,56.411012,12.351703,1.0,0.0,0.930858,False
4,4,9.649350,38.230000,14.113182,1.0,0.0,0.94875,False
...,...,...,...,...,...,...,...,...
726,726,10.475495,59.472229,22.242182,1.0,0.0,0.951075,False
727,727,10.419155,58.653487,10.341263,1.0,0.0,0.936763,False
728,728,10.551763,76.322151,8.237662,0.0,0.0,0.892279,True
729,729,10.778680,46.243342,23.974396,0.0,0.0,0.847549,True


In [14]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [24]:
from sklearn.preprocessing import StandardScaler
#Standardisierung

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [25]:
#Training

model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [28]:
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [30]:
print(r2_test)

0.6404797116982144


problem cannot turn datetime into float